In [15]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from config import username
from config import password
from config import host_number

In [2]:
# https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index.aspx
csv_file = "data/hpi_data.csv"
hpi_data_df = pd.read_csv(csv_file)
hpi_data_df.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00,100.00
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.98,101.06
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.33,100.95
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.72,101.02
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.35,101.40


In [8]:
#drop columns
hpi_data_clean = hpi_data_df[["hpi_type","hpi_flavor","frequency","level","place_name","place_id","yr","period","index_nsa"]]
# hpi_data_clean.head()

In [7]:
#level = location
# index nsa = price
#yr = year
#hpi flavor = purchase type
hpi_df = hpi_data_clean.rename(columns = {"level": "location", "index_nsa": "price", "yr":"year", "hpi_flavor":"purchase_type"})
hpi_df.head()

,hpi_type,purchase_type,frequency,location,place_name,place_id,year,period,price
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.98
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.33
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.72
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.35


In [10]:
#state dataframe (quarterly)
state_df = hpi_df.loc[hpi_df["location"] == "State"]
state_df.head()

,hpi_type,purchase_type,frequency,location,place_name,place_id,year,period,price
64331,traditional,all-transactions,quarterly,State,Alaska,AK,1975,1,62.74
64332,traditional,all-transactions,quarterly,State,Alaska,AK,1975,2,64.16
64333,traditional,all-transactions,quarterly,State,Alaska,AK,1975,3,62.24
64334,traditional,all-transactions,quarterly,State,Alaska,AK,1975,4,68.51
64335,traditional,all-transactions,quarterly,State,Alaska,AK,1976,1,71.31


In [12]:
#usa dataframe (monthly)
usa_df = hpi_df.loc[hpi_df["location"] == "USA or Census Division"]
usa_df.head()

,hpi_type,purchase_type,frequency,location,place_name,place_id,year,period,price
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.98
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.33
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.72
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.35


In [13]:
#puerto rico dataframe (quarterly)
pr_df = hpi_df.loc[hpi_df["location"] == "Puerto Rico"]
pr_df.head()

,hpi_type,purchase_type,frequency,location,place_name,place_id,year,period,price
114523,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1995,1,100.00
114524,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1995,2,102.34
114525,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1995,3,106.22
114526,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1995,4,111.45
114527,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1996,1,113.46


In [29]:
engine = create_engine("sqlite:///housingdata.db")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
print(Base.classes.keys())
# Save reference to the tables
state = Base.classes.state
usa = Base.classes.usa
puerto_rico = Base.classes.puerto_rico

['puerto_rico', 'state', 'usa']


In [30]:
state_df.to_sql(name='state', con=engine, if_exists='replace', index=False)

In [31]:
pr_df.to_sql(name='puerto_rico', con=engine, if_exists='replace', index=False)

In [32]:
usa_df.to_sql(name='usa', con=engine, if_exists='replace', index=False)